# Draw with ipycanvas

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adamelliotfields/ml/blob/main/voila/draw.ipynb)
[![Render nbviewer](https://img.shields.io/badge/render-nbviewer-f37726)](https://nbviewer.org/github/adamelliotfields/ml/blob/main/voila/draw.ipynb)

Drawing app built with `ipywidgets` and `ipycanvas`. Originally based on [examples/hand_drawing.ipynb](https://github.com/martinRenou/ipycanvas/blob/master/examples/hand_drawing.ipynb).

**Features**:
  * Resizable canvas
  * Line width slider
  * Stroke and fill color pickers
  * Fill shape checkbox
  * Filename text input
  * Save to file and clear buttons
  * Autosave to Numpy array
  * Debouncing

**Notes**:

⚠️ Works in Jupyter, not when rendered by Voila ⚠️

The background of the drawn image will be transparent. To add a background color, you'll want to use [`MultiCanvas`](https://ipycanvas.readthedocs.io/en/latest/api.html#ipycanvas.canvas.MultiCanvas) (layers). The first canvas will be filled and the second canvas will be what you draw on. Removing the background would just be clearing the bottom canvas.

In [ ]:
# @title Imports
import PIL
import threading

from ipycanvas import RoughCanvas, hold_canvas
from ipywidgets import (
    Button,
    Checkbox,
    ColorPicker,
    HBox,
    IntSlider,
    Text,
    VBox,
    link,
)

In [ ]:
# @title Functions
# https://github.com/salesforce/decorator-operations/blob/master/decoratorOperations/debounce_functions/debounce.py
def debounce(wait):
    def decorator(fn):
        def debounced(*args, **kwargs):
            def call_fn():
                debounced._timer = None
                return fn(*args, **kwargs)

            if debounced._timer is not None:
                debounced._timer.cancel()

            debounced._timer = threading.Timer(wait, call_fn)
            debounced._timer.start()

        debounced._timer = None
        return debounced

    return decorator


def on_mouse_down(x, y):
    global drawing, position, shape
    drawing = True
    position = (x, y)
    shape = [position]


def on_mouse_move(x, y):
    global position
    if not drawing or position is None:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        position = (x, y)

    shape.append(position)


def on_mouse_up(x, y):
    global drawing, fill_checkbox, position, shape
    if position is None:
        return

    with hold_canvas():
        canvas.stroke_line(position[0], position[1], x, y)
        if fill_checkbox.value:
            canvas.fill_polygon(shape)

    drawing = False
    shape = []


def on_save_click(_):
    global image, file_name_input
    if image is None or file_name_input.value == "":
        return
    img = PIL.Image.fromarray(image)
    img.save(file_name_input.value, format="PNG")  # always png


def on_clear_click(_):
    global canvas, image
    canvas.clear()
    image = None


@debounce(0.15)
def on_image_data(_):
    global canvas, image
    image = canvas.get_image_data()


@debounce(0.15)
def update_canvas_size(_):
    # note that updating the canvas size will clear the canvas
    global canvas, height_slider, width_slider
    canvas.width = width_slider.value
    canvas.height = height_slider.value


@debounce(0.15)
def update_canvas_thickness(_):
    global canvas, thickness_slider
    canvas.line_width = thickness_slider.value * 2

In [ ]:
# state
drawing = False
position = None
shape = []
image = None

default_width = 640
default_height = 480
default_thickness = 1

default_stroke_color = "#749cb8"
default_fill_color = "#c4d8e2"

canvas = RoughCanvas(width=default_width, height=default_height, sync_image_data=True)
canvas.layout.border = "1px solid black"  # border isn't included in image data
canvas.line_width = default_thickness

canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
canvas.observe(on_image_data, "image_data")

file_name_input = Text(value="image.png", description="File:")

stroke_picker = ColorPicker(description="Stroke:", value=default_stroke_color)
fill_picker = ColorPicker(description="Fill:", value=default_fill_color)
fill_checkbox = Checkbox(description="Fill?", value=False)
fill_checkbox.layout.width = "fit-content"

link((stroke_picker, "value"), (canvas, "stroke_style"))
link((fill_picker, "value"), (canvas, "fill_style"))

width_slider = IntSlider(description="Width:", value=default_width, min=256, max=1024)
height_slider = IntSlider(description="Height:", value=default_height, min=256, max=1024)
thickness_slider = IntSlider(
    description="Thickness:",
    value=default_thickness,
    min=1,
    max=5,
    step=1,
)

width_slider.observe(update_canvas_size, names="value")
height_slider.observe(update_canvas_size, names="value")
thickness_slider.observe(update_canvas_thickness, names="value")

save_button = Button(description="Save", button_style="success")
save_button.layout.width = "fit-content"
save_button.on_click(on_save_click)
save_button.layout.margin = "2px 0 0 auto"  # "margin-left: auto"

clear_button = Button(description="Clear", button_style="")
clear_button.layout.width = "fit-content"
clear_button.on_click(on_clear_click)

left_box = VBox(
    [
        file_name_input,
        stroke_picker,
        fill_picker,
        HBox([fill_checkbox, save_button, clear_button]),
    ]
)
right_box = VBox(
    [
        width_slider,
        height_slider,
        thickness_slider,
    ]
)

# render
VBox([HBox([canvas]), HBox([left_box, right_box])])